# UI - prva domaća zadaća: [Lovac na blago](http://degiorgi.math.hr/~singer/ui/ui_1415/zadace/lit/lovac_na_blago.pdf)

Dana je matrica dimenzija $10\cdot10$ u kojoj su, na nekim koordinatama, upisani podaci o broju novčića koji se nalaze u susjednim poljima. Za danu matricu želimo naći točne koordinate novčića.

Prvi korak pri rješavanju problema jest ustanoviti kako izgleda stablo pretraživanja. Svaki čvor stabla bit će jednak jednoj matrici u koju će biti upisani zadani brojevi novčića i označene lokacije zakopanih novčića. Uočavamo da je najjednostavniji način za razraditi stablo sljedeći:
*   Korijenski čvor je jednak originalnoj matrici;
*   U svakom čvoru djetetu tražimo lokaciju na koju se može dodati još jedan novčić sukladno pravilima lova na blago;
*   Ako detektiramo da su sva zadane upute zadovoljene, onda smo gotovi s pretraživanjem.

Konkretno, dano polje brojeva zapisat ćemo u matricu (tj. listu lista), s tim da će na praznim mjestima pisati $-1$. Prolazeći po stablu pretraživanja, na ta prazna mjesta upisivat ćemo $10$ ako tamo želimo smjestiti novčić, odnosno $-10$ ako detektiramo da se novčić tamo ne smije pojaviti.

Uočimo da zadana pravila mogu biti brojevi između $0$ i $8$, budući da pojedina ćelija ima najviše $8$ susjednih ćelija.

Navedimo neke funkcije koje ćemo moći koristiti za sve vrste pretraživanja. Za početak, lambde za pretvorbu stanja u tip $\texttt{list}$ ili $\texttt{tuple}$:

In [ ]:
to_list = lambda node: [list(row) for row in node]
to_tuple = lambda node: tuple(tuple(row) for row in node)

Pravila lova na blago nalažu da u praznu ćeliju ne možemo staviti novčić ukoliko u nekom od susjednih ćelija postoji uputa koja je već ispunjena, odnosno ako je polje s uputom okruženo točnim brojem novčića. Drugim riječima, ako neka susjedna ćelija sadrži broj između $0$ i $8$, a susjedne ćelije te ćelije ukupno sadrže točno taj broj novčića, onda naša ćelija nije dostupna za novi novčić. Isto tako, ako nijedna susjedna ćelija ne sadrži nikakvo pravilo, nemamo razloga u tu ćeliju staviti novčić. Sljedeća funkcija vraća broj pojavljivanja dane vrijednosti $\texttt{val}$ u susjednim ćelijama ćelije čvora $\texttt{node}$ na poziciji $(\texttt{row}, \texttt{col})$ (primjerice, ako nas zanima koliko novčića okružuje danu ćeliju, stavimo $\texttt{val}=10$):

In [ ]:
def num_of_sorrounding_values(node, row, col, val):
    num = 0
    n = len(node)
    directions = [ (1, 0), (-1, 0), (0, 1), (0, -1), (1, -1), (1, 1), (-1, 1), (-1, -1) ]
    for dx, dy in directions:
        if row + dx in range(n) and col + dy in range(n) and node[row + dx][col + dy] == val:
            num += 1
    return num

Nadalje, za potrebe označavanja ćelije $(\texttt{row}, \texttt{col})$ ćelijom koja sadrži ili ne smije sadržavati novčić, koristit ćemo funkciju za postavljanje vrijednosti na tim koordinatama:

In [ ]:
def set_value(node, row, col, val):
    node = to_list(node)
    node[row][col] = val
    return to_tuple(node)

Sljedeća funkcija vraća skup svih stanja koja mogu naslijediti dano stanje čvora $\texttt{node}$ ako u trenutno stanje dodamo točno jedan novčić (naravno, na neko od legalnih mjesta). Pretpostavljamo da će sva ilegalna mjesta biti označena prije poziva ove funkcije.

In [ ]:
def expand(node):
    n = len(node)
    new_nodes = set()
    for row in range(n):
        for col in range(n):
            if node[row][col] == -1:
                new_nodes.add(set_value(node, row, col, 10))
    return new_nodes

Koristit ćemo i funkciju za detektiranje je li dano stanje jednako ciljnom. Ako su sva pravila zadovoljena, onda je igra riješena.

In [ ]:
def search_done(node):
    n = len(node)
    for i in range(n):
        for j in range(n):
            if node[i][j] in range(9) and num_of_sorrounding_values(node, i, j, 10) != node[i][j]:
                return False
    return True

Kako bismo smanjili broj grana koje nastaju od svakog čvora, koristimo funkciju koja uzima čvor, detektira na koja mjesta nikako ne smije doći novčić i ta mjesta označava s $-10$; potom funkcija vraća taj prilagođeni čvor. Drugim riječima, ova funkcija označava ilegalna mjesta.

In [ ]:
def optimise(node):                 #označuje prazna mjesta na koja sigurno ne može doći novčić s -10
    n = len(node)
    directions = [ (1, 0), (-1, 0), (0, 1), (0, -1), (1, -1), (1, 1), (-1, 1), (-1, -1) ]
    for row in range(n):
        for col in range(n):
            if node[row][col] == -1:
                temp = 0
                for i in range(1, 9):
                    temp += num_of_sorrounding_values(node, row, col, i)
                if temp == 0:          #ako oko ćelije nema pravila po kojem bi tu bio novčić, onda tu ne smije biti novčić
                    node = set_value(node, row, col, -10)
                for dx, dy in directions:
                    if row + dx in range(n) and col + dy in range(n):               #ako je u okolini ćelije neko pravilo koje je ispunjeno, onda tu ne smije biti novčić
                        if node[row + dx][col + dy] in range(9) and num_of_sorrounding_values(node, row + dx, row + dy, 10) >= node[row + dx][col + dy]:
                            node = set_value(node, row, col, -10)
    return nodeLifoQueue(

Na kraju, dodajmo i metodu za ispis čvora. Na mjestu gdje stoji novčić ispisat ćemo N, na prazno mjesto samo točku, a pravila prepisujemo normalno.

In [ ]:
def print_node(node):
    n = len(node)
    for i in range(n):
        for j in range(n):
            temp = node[i][j]
            if temp in range(9):
                print(temp, end = ' ')
            elif temp <= 0:
                print('.', end = ' ')
            else:
                print('N', end = ' ')
        print('')

## Pretraživanje u širinu

Konačno, imamo sve spremno za implementaciju algoritma za pretraživanje u širinu. Kao što znamo, ovaj algoritam za strukutru podataka koristi red (queue). Kako bismo smanjili količinu grana u stablu, u red stavljamo samo optimizirane čvorove i otvaramo samo čvorove koji nisu već posjećeni.

In [ ]:
import queue

def breadth_first_search(root):
    open = queue.Queue()
    open.put(optimise(root))
    visited = set()
    while open.empty() == False:
        node = open.get()
        if node in visited:
            continue
        if search_done(node):
            return node
        for x in expand(node):
            if x not in visited:
                open.put(optimise(x))
        visited.add(node)
    n = len(root)
    return to_tuple([[-10] * n for _ in range(n)])              #ako ne nađemo dobro rješenje, vratit ćemo neku matricu napunjenu s -1

Slijede primjeri pokretanja BFS-a, od kojih su četiri moja, te dva prepisana iz dokumenta sa zadatkom.

In [ ]:
%%time
node = [[-1] * 4 for _ in range(4)]
node[0][0] = 0
node[0][2] = node[1][1] = 1
node[1][3] = node[2][0] = 2
node[2][1] = 4
print("poziv BFS na sljedećoj matrici:")
print_node(node)
print('\n')
new_node = breadth_first_search(node)
print('rezultat je:')
print_node(new_node)

In [ ]:
%%time
node = [[-1] * 6 for _ in range(6)]
node[0][0] = node[2][4] = node[5][1] = 0
node[0][2] = node[1][1] = node[4][4] = node[5][3] = 1
node[0][4] = node[1][3] = node[2][0] = 2
node[4][1] = 3
node[2][1] = 4
print("poziv BFS na sljedećoj matrici:")
print_node(node)
print('\n')
new_node = breadth_first_search(node)
print('rezultat je:')
print_node(new_node)

In [ ]:
%%time
node = [[-1] * 7 for _ in range(7)]
node[0][0] = node[2][4] = node[5][1] = node[2][6] = node[3][6] = node[6][0] = node[6][2] = node[6][4] = 0
node[0][2] = node[1][1] = node[3][4] = node[5][3] = node[1][6] = node[5][6] = 1
node[0][4] = node[1][3] = node[2][0] = node[4][3] = 2
node[4][1] = 3
node[2][1] = 4
print("poziv BFS na sljedećoj matrici:")
print_node(node)
print('\n')
new_node = breadth_first_search(node)
print('rezultat je:')
print_node(new_node)

In [ ]:
%%time
node = [[-1] * 8 for _ in range(8)]
for i in range(5):
    node[i][7] = 0
    node[7][i] = 0
node[0][0] = node[2][4] = node[5][1] = node[2][6] = node[3][6] = node[6][0] = node[6][2] = node[6][4] = 0
node[0][2] = node[1][1] = node[3][4] = node[5][3] = node[1][6] = node[5][6] = 1
node[0][4] = node[1][3] = node[2][0] = node[4][3] = 2
node[4][1] = 3
node[2][1] = 4
print("poziv BFS na sljedećoj matrici:")
print_node(node)
print('\n')
new_node = breadth_first_search(node)
print('rezultat je:')
print_node(new_node)

In [ ]:
%%time
#primjer (a)
node = [[-1] * 10 for _ in range(10)]
node[1][1] = node[2][1] = node[6][3] = 0
node[1][2] = node[2][3] = node[3][4] = node[7][4] = node[9][2] = 1
node[0][6] = node[0][8] = node[3][9] = node[4][0] = node[4][8] = node[5][6] = node[6][0] = node[7][0] = node[7][3] = node[9][1] = node[9][3] = node[9][8] = 2
node[1][5] = node[1][8] = node[4][7] = node[5][7] = node[7][6] = node[8][7] = node[9][6] = 3
node[1][6] = node[2][8] = node[5][2] = node[7][7] = node[7][8] = 4
print("poziv BFS na sljedećoj matrici:")
print_node(node)
print('\n')
new_node = breadth_first_search(node)
print('rezultat je:')
print_node(new_node)

In [ ]:
%%time
#primjer (b)
node = [[-1] * 10 for _ in range(10)]
node[1][0] = node[2][0] = node[2][7] = node[9][1] = 0
node[1][2] = node[1][4] = node[1][7] = node[1][9] = node[2][5] = node[2][9] = node[3][5] = node[4][0] = node[5][4] = node[5][9] = node[8][9] = 1
node[1][8] = node[2][3] = node[3][6] = node[6][0] = node[6][2] = node[7][7] = node[8][0] = 2
node[3][8] = node[4][3] = node[5][5] = node[5][7] = node[7][4] = node[7][6] = node[8][7] = 3
node[9][6] = 4
node[8][4] = 6
print("poziv BFS na sljedećoj matrici:")
print_node(node)
print('\n')
new_node = breadth_first_search(node)
print('rezultat je:')
print_node(new_node)depth_first

## Pretraživanje u dubinu

Vrlo slično implementiramo i pretraživanje u dubinu, samo umjesto reda koristimo stog. Python nema strukutru stog u obliku posebne strukture podataka, ali postoji struktura $\texttt{queue.LifoQueue}$, koja je u stvari LIFO red, odnosno red u kojem element koji je zadnji ubačen, zadnji iz njega izlazi. Naravno, takva strukutra ekvivalentna je stogu pa ju koristimo za implementaciju algoritma.

In [ ]:
def depth_first_search(root):                 #razlikuje se od BFS samo po strukturi 'open'!
    open = queue.LifoQueue()
    open.put(optimise(root))
    visited = set()
    while open.empty() == False:
        node = open.get()
        if node in visited:
            continue
        if search_done(node):
            return node
        for x in expand(node):
            if x not in visited:
                open.put(optimise(x))
        visited.add(node)
    n = len(root)
    return to_tuple([[-10] * n for _ in range(n)])              #ako ne nađemo dobro rješenje, vratit ćemo neku matricu napunjenu s -1

Pokrećemo iste primjere kao kod BFS-a:

In [ ]:
%%time
node = [[-1] * 4 for _ in range(4)]
node[0][0] = 0
node[0][2] = node[1][1] = 1
node[1][3] = node[2][0] = 2
node[2][1] = 4
print("poziv DFS na sljedećoj matrici:")
print_node(node)
print('\n')
new_node = depth_first_search(node)
print('rezultat je:')
print_node(new_node)

In [ ]:
%%time
node = [[-1] * 6 for _ in range(6)]
node[0][0] = node[2][4] = node[5][1] = 0
node[0][2] = node[1][1] = node[4][4] = node[5][3] = 1
node[0][4] = node[1][3] = node[2][0] = 2
node[4][1] = 3
node[2][1] = 4
print("poziv DFS na sljedećoj matrici:")
print_node(node)
print('\n')
new_node = depth_first_search(node)
print('rezultat je:')
print_node(new_node)

In [ ]:
%%time
node = [[-1] * 7 for _ in range(7)]
node[0][0] = node[2][4] = node[5][1] = node[2][6] = node[3][6] = node[6][0] = node[6][2] = node[6][4] = 0
node[0][2] = node[1][1] = node[3][4] = node[5][3] = node[1][6] = node[5][6] = 1
node[0][4] = node[1][3] = node[2][0] = node[4][3] = 2
node[4][1] = 3
node[2][1] = 4
print("poziv DFS na sljedećoj matrici:")
print_node(node)
print('\n')
new_node = depth_first_search(node)
print('rezultat je:')
print_node(new_node)

In [ ]:
%%time
node = [[-1] * 8 for _ in range(8)]
for i in range(5):
    node[i][7] = 0
    node[7][i] = 0
node[0][0] = node[2][4] = node[5][1] = node[2][6] = node[3][6] = node[6][0] = node[6][2] = node[6][4] = 0
node[0][2] = node[1][1] = node[3][4] = node[5][3] = node[1][6] = node[5][6] = 1
node[0][4] = node[1][3] = node[2][0] = node[4][3] = 2
node[4][1] = 3
node[2][1] = 4
print("poziv DFS na sljedećoj matrici:")
print_node(node)
print('\n')
new_node = depth_first_search(node)
print('rezultat je:')
print_node(new_node)

In [ ]:
%%time
#primjer (a)
node = [[-1] * 10 for _ in range(10)]
node[1][1] = node[2][1] = node[6][3] = 0
node[1][2] = node[2][3] = node[3][4] = node[7][4] = node[9][2] = 1
node[0][6] = node[0][8] = node[3][9] = node[4][0] = node[4][8] = node[5][6] = node[6][0] = node[7][0] = node[7][3] = node[9][1] = node[9][3] = node[9][8] = 2
node[1][5] = node[1][8] = node[4][7] = node[5][7] = node[7][6] = node[8][7] = node[9][6] = 3
node[1][6] = node[2][8] = node[5][2] = node[7][7] = node[7][8] = 4
print("poziv DFS na sljedećoj matrici:")
print_node(node)
print('\n')
new_node = depth_first_search(node)
print('rezultat je:')
print_node(new_node)

In [ ]:
%%time
#primjer (b)
node = [[-1] * 10 for _ in range(10)]
node[1][0] = node[2][0] = node[2][7] = node[9][1] = 0
node[1][2] = node[1][4] = node[1][7] = node[1][9] = node[2][5] = node[2][9] = node[3][5] = node[4][0] = node[5][4] = node[5][9] = node[8][9] = 1
node[1][8] = node[2][3] = node[3][6] = node[6][0] = node[6][2] = node[7][7] = node[8][0] = 2
node[3][8] = node[4][3] = node[5][5] = node[5][7] = node[7][4] = node[7][6] = node[8][7] = 3
node[9][6] = 4
node[8][4] = 6
print("poziv DFS na sljedećoj matrici:")
print_node(node)
print('\n')
new_node = depth_first_search(node)
print('rezultat je:')
print_node(new_node)

##A* pretraživanje

Prethodni algoritmi pretraživanja nisu uzimali u obzir cijenu kopanja novčića. Međutim, ovaj algoritam koristi i stvarnu cijenu iskopa, kao i neku heuristiku kao procjenu koliko kopanja je potrebno od trenutnog do ciljnog stanja.


Za početak, uvedimo funkciju koja vraća ukupan broj novčića u danom čvoru. Naime, broj novčića u trenutnom čvoru je točno jednak broju koraka napravljenih od startnog stanja do trenutnog, budući da se svaki korak (tj. prijelaz iz roditeljskog čvora u dijete) sastoji od dodavanja točno jednog novčića.

In [ ]:
def total_coins(node):
    n = len(node)
    num = 0
    for row in range(n):
        for col in range(n):LifoQueue(
            if node[row][col] == 10:
                num += 1
    return num

Heuristika nam može raditi na način da nas usmjerava tako da stavljamo novčiće na mjesta kod kojih ima najviše uputa (odnosno najviše brojeva između $0$ i $8$ koji određuju poziciju novčića). Ovo možemo postići na način da promatramo ukupan zbroj takvih brojeva na ploči, te svaki put kad dodamo novčić negdje, umanjimo taj ukupni zbroj. Drugim riječima, kad je novčić dodan, susjedna mu pravila smiju dobiti za jedan manje dodatnih novčića.

Formalizirajmo ovo. Neka je $\texttt{State}$ neko stanje igre. Tada je heuristika $h$ dana s:
$$h(\texttt{State})=\sum_{\quad\quad i,j\\ \texttt{State}[i][j]\in[0,8]}(\texttt{State}[i][j]-\texttt{num_of_sorrounding_vals}(\texttt{State}, i, j, 10)). $$

Ova heuristika je dobro definirana jer je monotono nerastuća kako se približavamo ciljnom stanju; vrijednost heuristike u ciljnom stanju iznosi $0$.

Međutim, uočavamo da ova heuristika nažalost nije optimistična. Naime, već u korijenskom čvoru vidimo da zbroj uputa može biti veći ili jednak ukupnom broju novčića, dakle heuristika precjenjuje broj koraka do cilja. Ipak, čak i uz korištenje neoptimalne heuristike, pretraživanje radi brže od BFS-a i DFS-a.

In [ ]:
def heuristic(node):
    n = len(node)
    num = 0
    for row in range(n):
        for col in range(n):
            if node[row][col] in range(9):
                num += node[row][col] - num_of_sorrounding_values(node, row, col, 10)
    return num

Napokon, implementirajmo i A* algoritam za pretraživanje:

In [ ]:
import heapq

def a_star_search(root):
    root = optimise(root)
    open = []
    heapq.heappush(open, (heuristic(root), root))
    visited = set()
    while len(open) > 0:
        temp = heapq.heappop(open)
        priority = temp[0]
        node = temp[1]
        if node in visited:
            continue
        if search_done(node):
            return node
        for x in expand(node):
            x = optimise(x)
            if x not in visited:
                temp = heuristic(x) + total_coins(x)
                heapq.heappush(open, (temp, x))
        visited.add(node)
    return to_tuple([[-10] * n for _ in range(n)])              #ako ne nađemo dobro rješenje, vratit ćemo neku matricu napunjenu s -1

Koristimo iste primjere kao ranije za pokretanje:

In [ ]:
%%time
node = [[-1] * 4 for _ in range(4)]
node[0][0] = 0
node[0][2] = node[1][1] = 1
node[1][3] = node[2][0] = 2
node[2][1] = 4
print("poziv A* na sljedećoj matrici:")
print_node(node)
print('\n')
new_node = a_star_search(node)
print('rezultat je:')
print_node(new_node)

In [ ]:
%%time
node = [[-1] * 6 for _ in range(6)]
node[0][0] = node[2][4] = node[5][1] = 0
node[0][2] = node[1][1] = node[4][4] = node[5][3] = 1
node[0][4] = node[1][3] = node[2][0] = 2
node[4][1] = 3
node[2][1] = 4
print("poziv A* na sljedećoj matrici:")
print_node(node)
print('\n')
new_node = a_star_search(node)
print('rezultat je:')
print_node(new_node)

In [ ]:
%%time
node = [[-1] * 7 for _ in range(7)]
node[0][0] = node[2][4] = node[5][1] = node[2][6] = node[3][6] = node[6][0] = node[6][2] = node[6][4] = 0
node[0][2] = node[1][1] = node[3][4] = node[5][3] = node[1][6] = node[5][6] = 1
node[0][4] = node[1][3] = node[2][0] = node[4][3] = 2
node[4][1] = 3
node[2][1] = 4
print("poziv A* na sljedećoj matrici:")
print_node(node)
print('\n')
new_node = a_star_search(node)
print('rezultat je:')
print_node(new_node)

In [ ]:
%%time
node = [[-1] * 8 for _ in range(8)]
for i in range(5):
    node[i][7] = 0
    node[7][i] = 0
node[0][0] = node[2][4] = node[5][1] = node[2][6] = node[3][6] = node[6][0] = node[6][2] = node[6][4] = 0
node[0][2] = node[1][1] = node[3][4] = node[5][3] = node[1][6] = node[5][6] = 1
node[0][4] = node[1][3] = node[2][0] = node[4][3] = 2
node[4][1] = 3
node[2][1] = 4
print("poziv A* na sljedećoj matrici:")
print_node(node)
print('\n')
new_node = a_star_search(node)
print('rezultat je:')
print_node(new_node)

In [ ]:
%%time
#primjer (a)
node = [[-1] * 10 for _ in range(10)]
node[1][1] = node[2][1] = node[6][3] = 0
node[1][2] = node[2][3] =node[3][4] =node[7][4] =node[9][2] = 1
node[0][6] =node[0][8] =node[3][9] =node[4][0] =node[4][8] =node[5][6] = node[6][0] = node[7][0] = node[7][3] = node[9][1] = node[9][3] = node[9][8] = 2
node[1][5] = node[1][8] = node[4][7] = node[5][7] = node[7][6] = node[8][7] = node[9][6] = 3
node[1][6] = node[2][8] = node[5][2] = node[7][7] = node[7][8] = 4
print("poziv A* na sljedećoj matrici:")
print_node(node)
print('\n')
new_node = a_star_search(node)
print('rezultat je:')
print_node(new_node)

In [ ]:
%%time
#primjer (b)
node = [[-1] * 10 for _ in range(10)]
node[1][0] = node[2][0] = node[2][7] = node[9][1] = 0
node[1][2] = node[1][4] = node[1][7] = node[1][9] = node[2][5] = node[2][9] = node[3][5] = node[4][0] = node[5][4] = node[5][9] = node[8][9] = 1
node[1][8] = node[2][3] = node[3][6] = node[6][0] = node[6][2] = node[7][7] = node[8][0] = 2
node[3][8] = node[4][3] = node[5][5] = node[5][7] = node[7][4] = node[7][6] = node[8][7] = 3
node[9][6] = 4
node[8][4] = 6
print("poziv A* na sljedećoj matrici:")
print_node(node)
print('\n')
new_node = a_star_search(node)
print('rezultat je:')
print_node(new_node)